# Entity Extraction+Linking using modern SciSpaCy models

In [1]:
import dask.dataframe as dd
import json
import numpy as np
import pandas as pd
import s3fs
import spacy
import scispacy

from dask.distributed import Client, progress, get_worker

from scispacy.linking_utils import RxNorm

In [2]:
# MODEL_KB = "umls"
# MODEL_KB = "mesh"
# MODEL_KB = "go"
# MODEL_KB = "hpo"
MODEL_KB = "rxnorm"

In [3]:
BUCKET_NAME = "saturn-elsevierinc"

SENTENCE_FOLDER = "/".join(["s3:/", BUCKET_NAME, "cord19-sents-pq-sm"])
ENTITIES_FOLDER = "/".join(["s3:/", BUCKET_NAME, 
                            "cord19-ents-{:s}-pq-sm".format(MODEL_KB)])

In [4]:
sentences_df = dd.read_parquet(SENTENCE_FOLDER, engine="pyarrow")
sentences_df.head(npartitions=10)

,cord_uid,pid,sid,stext
6498,sz7qmi8q,A0,0,Schwer punkt: Lun gen-und Pleura pa tho lo gie...
6498,sz7qmi8q,A0,1,für Pa tho lo gie der Ruhr-Uni ver si tät Bo c...
6498,sz7qmi8q,A0,2,fi ka ti on der Er kran kun gen des pul mo
6498,sz7qmi8q,A0,3,na len Sur fac tant-Sys tems TYP I TYP II TYP ...
6498,sz7qmi8q,A0,4,re nenund Säug lings al ter Neu ge bo


In [5]:
len(sentences_df)

7313

## Processing

In [6]:
client = Client(processes=False, n_workers=1, threads_per_worker=1)
client

Client Scheduler: inproc://10.0.28.17/137/1 Dashboard: http://10.0.28.17:8787/status,Cluster Workers: 1 Cores: 1 Memory: 16.25 GB


In [7]:
def build_nlp_pipeline(model_kb):
    nlp = spacy.load("en_core_sci_md")
    kb = RxNorm()
    return nlp, kb


def nlp_workers():
    import traceback
    try:
        worker = get_worker()
        nlp, kb = build_nlp_pipeline(MODEL_KB)
        worker.nlp = nlp
        worker.kb = kb
    except:
        return traceback.format_exc()
    return 0


def check_nlp_workers():
    worker = get_worker()
    return str(worker.nlp)


%time client.run(nlp_workers)

CPU times: user 7.32 s, sys: 532 ms, total: 7.85 s
Wall time: 8.09 s


{'inproc://10.0.28.17/137/3': 0}

In [8]:
client.run(check_nlp_workers)

{'inproc://10.0.28.17/137/3': '<spacy.lang.en.English object at 0x7f9592f60ed0>'}

In [9]:
def handle_batch(sents, nlp, kb, model_kb):
    docs = nlp.pipe(sents, n_threads=16, batch_size=len(sents))
    ents_list = []
    for doc in docs:
        ents = []
        for eid, ent in enumerate(doc.ents):
            try:
                cuis = kb.alias_to_cuis[ent.text]
                for cui in cuis:
                    ents.append((eid, model_kb, ent.text, cui,
                                 1.0, ent.start_char, ent.end_char))
            except KeyError:
                continue
        ents_list.append(ents)
    return ents_list


def handle_partition(part):
    worker = get_worker()
    nlp, kb = worker.nlp, worker.kb
    batch_size = 32
    sent_batch, ent_batch, entities = [], [], []
    for _, row in part.iterrows():
        if len(sent_batch) % batch_size == 0 and len(sent_batch) > 0:
            ent_batch = handle_batch(sent_batch, nlp, kb, MODEL_KB)
            entities.extend(ent_batch)
            sent_batch = []
        try:
            sent_batch.append(row.stext)
        except ValueError:
            continue
    if len(sent_batch) > 0:
        ent_batch = handle_batch(sent_batch, nlp, kb, MODEL_KB)
        entities.extend(ent_batch)
    return entities

In [10]:
entities_df = sentences_df.copy()

In [11]:
entities_df["entities"] = entities_df.map_partitions(
    lambda part: handle_partition(part), meta=("object"))
entities_df = entities_df.drop(columns=["stext"])
entities_df = entities_df.explode("entities")
entities_df = entities_df.dropna()

entities_df["eid"] = entities_df.apply(
    lambda row: row.entities[0], meta=("int"), axis=1)
entities_df["eclass"] = entities_df.apply(
    lambda row: row.entities[1], meta=("str"), axis=1)
entities_df["etext"] = entities_df.apply(
    lambda row: row.entities[2], meta=("str"), axis=1)
entities_df["elabel"] = entities_df.apply(
    lambda row: row.entities[3], meta=("str"), axis=1)
entities_df["escore"] = entities_df.apply(
    lambda row: row.entities[4], meta=("float"), axis=1)
entities_df["ent_start_char"] = entities_df.apply(
    lambda row: row.entities[5], meta=("int"), axis=1)
entities_df["ent_end_char"] = entities_df.apply(
    lambda row: row.entities[6], meta=("int"), axis=1)

entities_df = entities_df.drop(columns=["entities"])

In [12]:
entities_df.cord_uid = entities_df.cord_uid.astype(str)
entities_df.pid = entities_df.pid.astype(str)
entities_df.sid = entities_df.sid.astype(np.int32)
entities_df.eid = entities_df.eid.astype(np.int32)
entities_df.eclass = entities_df.eclass.astype(str)
entities_df.etext = entities_df.etext.astype(str)
entities_df.elabel = entities_df.elabel.astype(str)
entities_df.escore = entities_df.escore.astype(np.float32)
entities_df.ent_start_char = entities_df.ent_start_char.astype(np.int32)
entities_df.ent_end_char = entities_df.ent_end_char.astype(np.int32)

In [13]:
fs = s3fs.S3FileSystem()
if fs.exists(ENTITIES_FOLDER):
    fs.rm(ENTITIES_FOLDER, recursive=True)

In [14]:
entities_df.to_parquet(ENTITIES_FOLDER, engine="pyarrow", compression="snappy")

## Verify Result

In [15]:
entities_df = dd.read_parquet(ENTITIES_FOLDER, engine="pyarrow")
entities_df.head(npartitions=10)

,cord_uid,pid,sid,eid,eclass,etext,elabel,escore,ent_start_char,ent_end_char
6498,sz7qmi8q,B12,4,7,rxnorm,BPD,C0006012,1.0,142,145
6498,sz7qmi8q,B13,0,0,rxnorm,BPD,C0006012,1.0,56,59
10017,xttko097,B10,6,0,rxnorm,Muscle stiffness,C0221170,1.0,0,16
10017,xttko097,B15,4,0,rxnorm,Muscle stiffness,C0221170,1.0,0,16
4931,gzts5h0y,B10,3,0,rxnorm,Fever,C0015967,1.0,0,5


In [16]:
len(entities_df)

58